In [1]:
#practice on class 
class Student:
    def __init__ (self,name,rollno):
        self.name=name
        self.rollno=rollno

    def display_info(self):
        print('Name Of Student :',self.name)
        print(f'{self.name} your roll no is : {self.rollno}')

    def update_rollno(self,new_rollno):
        self.rollno=new_rollno

In [2]:
s1=Student('kartik',20)

In [3]:
s1.name

'kartik'

In [4]:
s1.display_info()

Name Of Student : kartik
kartik your roll no is : 20


In [5]:
s1.update_rollno(25)
s1.display_info()

Name Of Student : kartik
kartik your roll no is : 25


In [6]:
import numpy as np
import pandas as pd 


In [7]:
# Excersize questions for that first load the data and set the pipeline for transformation 

# load csv file
housing=pd.read_csv('datasets/housing/housing.csv')
housing.head()

# train test split 
from sklearn.model_selection import train_test_split

housing_without_label=housing.drop('median_house_value',axis=1)
housing_label=housing['median_house_value']

x_train,x_test,y_train,y_test=train_test_split(housing_without_label,housing_label,test_size=0.2,random_state=42)

#pipeline 

from sklearn.pipeline import Pipeline,make_pipeline

num_col_names = ['longitude','latitude','housing_median_age','median_income']
cat_col_names = ['ocean_proximity']


# ❶ numerical & categorical pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
num_pip=make_pipeline(SimpleImputer(strategy='median'),StandardScaler())

from sklearn.preprocessing import OneHotEncoder
cat_pip=make_pipeline(SimpleImputer(strategy='most_frequent'),OneHotEncoder(sparse_output=False))

num_pip.fit_transform(housing[num_col_names]).shape
cat_pip.fit_transform(housing[cat_col_names])

# ❷ functions for column ratio 

def column_ratio(df):
    return (df[:,[0]]/df[:,[1]])

def naming(transformer, feature_names_in):
    return ['ratio']

# ❸ ratio and log pipeline

from sklearn.preprocessing import FunctionTransformer
ratio_pip=make_pipeline(SimpleImputer(strategy='median'),FunctionTransformer(column_ratio,feature_names_out=naming),
                        StandardScaler())

log_pip=make_pipeline(SimpleImputer(strategy='median'),FunctionTransformer(np.log,feature_names_out='one-to-one'))

# ❹ finally column trasformer for all the tasks

from sklearn.compose import ColumnTransformer
processing=column_transformer=ColumnTransformer([('num_pip',num_pip,num_col_names),
    ('cat_pip',cat_pip,cat_col_names),
    ('rooms_per_house',ratio_pip,['total_rooms','households']),
    ('bedroom',ratio_pip,['total_bedrooms','households']),
    ('people_per_house',ratio_pip,['population','households']),
    ('log_pip',log_pip,["total_bedrooms", "total_rooms", "population",
                           "households", "median_income"]),

],remainder=num_pip)

processing.fit_transform(housing_without_label)

print(processing.get_feature_names_out())

['num_pip__longitude' 'num_pip__latitude' 'num_pip__housing_median_age'
 'num_pip__median_income' 'cat_pip__ocean_proximity_<1H OCEAN'
 'cat_pip__ocean_proximity_INLAND' 'cat_pip__ocean_proximity_ISLAND'
 'cat_pip__ocean_proximity_NEAR BAY' 'cat_pip__ocean_proximity_NEAR OCEAN'
 'rooms_per_house__ratio' 'bedroom__ratio' 'people_per_house__ratio'
 'log_pip__total_bedrooms' 'log_pip__total_rooms' 'log_pip__population'
 'log_pip__households' 'log_pip__median_income']


In [8]:
# Q1
#Try a support vector machine regressor (sklearn.svm.SVR) with various hyperparameters, such as kernel="linear" (with various values for 
#the C hyperparameter) or kernel="rbf" (with various values for the C and gamma hyperparameters). Note that support vector machines don’t 
#scale well to large datasets, so you should probably train your model on just the first 5,000 instances of the training set and use only 
#3-fold cross-validation, or else it will take hours. Don’t worry about what the hyperparameters mean for now; we’ll discuss them in Chapter5
#How does the best SVR predictor perform?

from sklearn.svm import SVR

svr_pip=make_pipeline(processing,SVR())

svr_pip.fit(x_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                              SimpleImputer(strategy='median')),
                                                             ('standardscaler',
                                                              StandardScaler())]),
                                   transformers=[('num_pip',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['longitude', 'latitude',
                                                   'housing_median_age',
                                                   'medi...
                                                                                       func=<function column_ratio at 0x7767e39c3e20>)),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['population', 'households']),
                                                 ('log_pip',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('functiontransformer',
                                                                   FunctionTransformer(feature_names_out='one-to-one',
                                                                                       func=<ufunc 'log'>))]),
                                                  ['total_bedrooms',
                                                   'total_rooms', 'population',
                                                   'households',
                                                   'median_income'])])),
                ('svr', SVR())])

In [9]:
y_pred=svr_pip.predict(x_train)

In [10]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_train,y_pred,squared=False)

117780.19280474613

In [11]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'svr__C': [0.1, 1, 10],
    'svr__epsilon': [0.1, 0.2, 0.5],
    'svr__gamma': ['scale', 0.01, 0.1],
    'svr__kernel':['linear', 'poly', 'rbf', 'sigmoid']
}

grid_search=GridSearchCV(svr_pip,param_grid,scoring='neg_root_mean_squared_error',n_jobs=-1,cv=3)

In [12]:
grid_search.fit(x_test,y_test)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('num_pip',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude...
                                                                                          FunctionTransformer(feature_names_out='one-to-one',
                                                                                                              func=<ufunc 'log'>))]),
                                                                         ['total_bedrooms',
                                                                          'total_rooms',
                                                                          'population',
                                                                          'households',
                                                                          'median_income'])])),
                                       ('svr', SVR())]),
             n_jobs=-1,
             param_grid={'svr__C': [0.1, 1, 10],
                         'svr__epsilon': [0.1, 0.2, 0.5],
                         'svr__gamma': ['scale', 0.01, 0.1],
                         'svr__kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             scoring='neg_root_mean_squared_error')

In [13]:
grid_search.best_params_
pd.DataFrame(grid_search.cv_results_).sort_values(by='mean_test_score',ascending=False).head(3)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svr__C,param_svr__epsilon,param_svr__gamma,param_svr__kernel,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
105,0.711284,0.113937,0.260381,0.077768,10,0.5,0.1,poly,"{'svr__C': 10, 'svr__epsilon': 0.5, 'svr__gamm...",-68595.762210,-68668.331780,-73333.944103,-70199.346031,2216.693542,1
93,0.776260,0.116262,0.271308,0.040281,10,0.2,0.1,poly,"{'svr__C': 10, 'svr__epsilon': 0.2, 'svr__gamm...",-68595.840515,-68668.379598,-73333.840988,-70199.353701,2216.615046,2
81,0.832225,0.177498,0.279554,0.067314,10,0.1,0.1,poly,"{'svr__C': 10, 'svr__epsilon': 0.1, 'svr__gamm...",-68595.866643,-68668.395521,-73333.806596,-70199.356254,2216.588869,3


In [14]:
# final_svr_model=grid_search.best_estimator_
from sklearn.model_selection import RandomizedSearchCV

In [15]:
# Q2.Try replacing the GridSearchCV with a RandomizedSearchCV.

from sklearn.model_selection import RandomizedSearchCV


param_dist={'random_search__kernel':['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
            'random_search__C':[0,5,10,15,20,25,30],
            'random_search__epsilon':[5,10,15,20],
            'random_search__gamma':[0.1,0.4,0.7,1,3]}

from sklearn.pipeline import Pipeline
rand_pip=Pipeline([('processing',processing),('random_search',SVR())])

random_search=RandomizedSearchCV(rand_pip,param_dist,scoring='neg_root_mean_squared_error',cv=3,random_state=42,n_jobs=-1,n_iter=5)
random_search.fit(x_test,y_test)

/home/kartiksavaliya/anaconda3/envs/homl3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
3 fits failed out of a total of 15.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/home/kartiksavaliya/anaconda3/envs/homl3/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/kartiksavaliya/anaconda3/envs/homl3/lib/python3.10/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/kartiksavaliya/anaconda3/envs/homl3/lib/python3.10/site-packages/sklearn/

RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('processing',
                                              ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                           SimpleImputer(strategy='median')),
                                                                                          ('standardscaler',
                                                                                           StandardScaler())]),
                                                                transformers=[('num_pip',
                                                                               Pipeline(steps=[('simpleimputer',
                                                                                                SimpleImputer(strategy='median')),
                                                                                               ('standardscaler',
                                                                                                StandardScaler())]),
                                                                               ['longitude',
                                                                                'latitude'...
                                                                                'population',
                                                                                'households',
                                                                                'median_income'])])),
                                             ('random_search', SVR())]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'random_search__C': [0, 5, 10, 15, 20,
                                                             25, 30],
                                        'random_search__epsilon': [5, 10, 15,
                                                                   20],
                                        'random_search__gamma': [0.1, 0.4, 0.7,
                                                                 1, 3],
                                        'random_search__kernel': ['linear',
                                                                  'poly', 'rbf',
                                                                  'sigmoid',
                                                                  'precomputed']},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [16]:
# Q3) Try adding a SelectFromModel transformer in the preparation pipeline to select only the most important attributes.
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LinearRegression

feature_selection_pip=make_pipeline(processing,SelectFromModel(LinearRegression(),threshold='median'))

feature_selection_pip.fit_transform(x_train,y_train)

array([[ 1.17299302, -1.35041487,  0.37969797, ...,  8.6996814 ,
         7.88382321,  6.97728134],
       [ 1.26802809, -1.37853628,  0.42006824, ...,  6.80350526,
         6.5482191 ,  5.11799381],
       [-1.3529389 ,  0.98834939, -0.09231969, ...,  8.84058019,
         7.99766313,  7.30249642],
       ...,
       [ 0.11760365,  0.30406165, -0.91152173, ...,  7.87245515,
         6.98193468,  6.25382881],
       [ 1.18799856, -0.72705686, -0.63438241, ...,  7.81075812,
         7.35691824,  6.20253552],
       [ 0.35269038, -0.66144022,  1.26207687, ...,  7.11476945,
         6.51174533,  5.27299956]])

In [17]:
# Q5) Automatically explore some preparation options using GridSearchCV

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV

svr_pip=Pipeline([('processing',processing),('model',SVR(C=10,epsilon=0.5,gamma=0.1,kernel='poly'))])

param_grid = {
    'processing__num_pip__simpleimputer__strategy': ['median', 'mean'],
    'processing__log_pip__simpleimputer__strategy': ['median', 'mean'],
    'processing__rooms_per_house__simpleimputer__strategy': ['median', 'mean'],
    'processing__bedroom__simpleimputer__strategy': ['median', 'mean'],
    'processing__people_per_house__simpleimputer__strategy': ['median', 'mean'],
    'processing__cat_pip__simpleimputer__strategy': ['most_frequent', 'constant'],
    'processing__cat_pip__onehotencoder__handle_unknown': ['ignore', 'error']
}


grid_search=GridSearchCV(svr_pip,param_grid,scoring='neg_root_mean_squared_error',cv=3,n_jobs=-1)
grid_search.fit(x_test,y_test)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('processing',
                                        ColumnTransformer(remainder=Pipeline(steps=[('simpleimputer',
                                                                                     SimpleImputer(strategy='median')),
                                                                                    ('standardscaler',
                                                                                     StandardScaler())]),
                                                          transformers=[('num_pip',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardscaler',
                                                                                          StandardScaler())]),
                                                                         ['longitude',
                                                                          'latitude',
                                                                          'hous...
                         'processing__cat_pip__simpleimputer__strategy': ['most_frequent',
                                                                          'constant'],
                         'processing__log_pip__simpleimputer__strategy': ['median',
                                                                          'mean'],
                         'processing__num_pip__simpleimputer__strategy': ['median',
                                                                          'mean'],
                         'processing__people_per_house__simpleimputer__strategy': ['median',
                                                                                   'mean'],
                         'processing__rooms_per_house__simpleimputer__strategy': ['median',
                                                                                  'mean']},
             scoring='neg_root_mean_squared_error')

In [18]:
grid_search.best_params_

{'processing__bedroom__simpleimputer__strategy': 'median',
 'processing__cat_pip__onehotencoder__handle_unknown': 'ignore',
 'processing__cat_pip__simpleimputer__strategy': 'most_frequent',
 'processing__log_pip__simpleimputer__strategy': 'mean',
 'processing__num_pip__simpleimputer__strategy': 'median',
 'processing__people_per_house__simpleimputer__strategy': 'median',
 'processing__rooms_per_house__simpleimputer__strategy': 'median'}

In [19]:
grid_search.best_score_

-70172.13021401025

In [20]:
#Q6)Try to implement the StandardScalerClone class again from scratch,then add support for the inverse_transform() method: executing scaler.
# inverse_transform(scaler.fit_transform(X)) should return an array veryclose to X. Then add support for feature names: set 
# feature_names_in_ in the fit() method if the input is a DataFrame. This attribute should be a NumPy array of column names. Lastly, 
# implement the get_feature_names_out() method: it should have one optional input_features=None argument. If passed, the method should check
# that its length matches n_features_in_, and it should match feature_names_in_ if it is defined; then input_features should be
# returned. If input_features is None, then the method should either return feature_names_in_ if it is defined or np.array(["x0", "x1", ...]) 
# with length n_features_in_ otherwise.

from sklearn.base import BaseEstimator,TransformerMixin
from sklearn.base import check_array,check_is_fitted

class StandardScalerClone(BaseEstimator,TransformerMixin):
    def __init__(self,with_mean=True):
        self.with_mean=with_mean

    def fit(self,X,y=None):
        assert isinstance(X,pd.DataFrame)
        self.feature_names_in_=X.columns
        X=check_array(X)
        self.mean_=X.mean(axis=0)
        self.std_=X.std(axis=0)
        self.n_feature_in_=X.shape[1]
        self.std_[self.std_==0]=1
        return self

    def transform(self,X):
        check_is_fitted(self)
        X=check_array(X)
        assert X.shape[1]==self.n_feature_in_
        

        if self.with_mean:
            X=X-self.mean_
        return X/self.std_

    def inverse_transform(self,X):
        X=X*self.std_
        if self.with_mean:
            X=X+self.mean_
        return X

    def get_feature_names_out(self,input_feature=None):
        if input_feature!=None:
            assert self.n_feature_in_==len(input_feature)
            return input_feature
        else:
            if len(self.feature_names_in_)!=0:
                return self.feature_names_in_
            else:
                default_feature_names=[]
                for i in range(0,self.n_feature_in_):
                    v=f'x{i}'
                    default_feature_names.append(v)
                return default_feature_names

In [21]:
standard_scaler=StandardScalerClone()

standard_scaler.fit(housing[num_col_names])

StandardScalerClone()

In [22]:
standard_scaler.n_feature_in_
standard_scaler.feature_names_in_
standard_scaler.get_feature_names_out()

Index(['longitude', 'latitude', 'housing_median_age', 'median_income'], dtype='object')

In [23]:
t=standard_scaler.transform(housing[num_col_names])
print(t)

[[-1.32783522  1.05254828  0.98214266  2.34476576]
 [-1.32284391  1.04318455 -0.60701891  2.33223796]
 [-1.33282653  1.03850269  1.85618152  1.7826994 ]
 ...
 [-0.8237132   1.77823747 -0.92485123 -1.14259331]
 [-0.87362627  1.77823747 -0.84539315 -1.05458292]
 [-0.83369581  1.75014627 -1.00430931 -0.78012947]]


In [24]:
standard_scaler.mean_
standard_scaler.std_[standard_scaler.std_==0]=1


In [25]:
result=standard_scaler.inverse_transform(t)
result

array([[-122.23  ,   37.88  ,   41.    ,    8.3252],
       [-122.22  ,   37.86  ,   21.    ,    8.3014],
       [-122.24  ,   37.85  ,   52.    ,    7.2574],
       ...,
       [-121.22  ,   39.43  ,   17.    ,    1.7   ],
       [-121.32  ,   39.43  ,   18.    ,    1.8672],
       [-121.24  ,   39.37  ,   16.    ,    2.3886]])

In [26]:
housing[num_col_names]

,longitude,latitude,housing_median_age,median_income
0,-122.23,37.88,41.0,8.3252
1,-122.22,37.86,21.0,8.3014
2,-122.24,37.85,52.0,7.2574
3,-122.25,37.85,52.0,5.6431
4,-122.25,37.85,52.0,3.8462
...,...,...,...,...
20635,-121.09,39.48,25.0,1.5603
20636,-121.21,39.49,18.0,2.5568
20637,-121.22,39.43,17.0,1.7000
20638,-121.32,39.43,18.0,1.8672
